In [46]:
import sys
from functools import partial

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import wandb
from ignite.engine import (Engine, Events, create_supervised_evaluator,
                           create_supervised_trainer)
from ignite.metrics import Accuracy, Loss
from scipy.io.arff import loadarff
from sklearn.model_selection import train_test_split
from torch import nn
from torch.functional import F
from torch.utils.data import DataLoader, Dataset, SubsetRandomSampler

sys.path.append('../')
from src.datasets import FordDataset
from src.models import TransformerClassification
from src.utils import split_batch, build_optimizer

In [47]:
n_splits = 50

config = {
    "model":{
        "encoder": {
            "d_model": n_splits,
            "nhead": n_splits // 2,
            "layer_norm_eps": 1e-3,
            "dropout": 0.5
        },
        "num_layers": 2,
        "fc":{
            "dim": 50,
            "dropout": 0.5
        }
    },
    "train":{
        "optimizer": torch.optim.Adam,
        "lr": 3e-4,
        "n_epoch": 30
    },
    "random_state": np.random.randint(0, 1000)
}

In [48]:
train_path = "../data/FordA/FordA_TRAIN.arff"
test_path = "../data/FordA/FordA_TEST.arff"

train_dataset = FordDataset(train_path)
test_dataset = FordDataset(test_path)

idx = np.arange(len(train_dataset))
idx_train, idx_val = train_test_split(idx, train_size=0.8, stratify=train_dataset.labels, random_state=config['random_state'])

train_sampler = SubsetRandomSampler(idx_train)
val_sampler = SubsetRandomSampler(idx_val)

train_dataloader = DataLoader(train_dataset, batch_size=128, sampler=train_sampler, collate_fn=partial(split_batch, n_splits=n_splits))
val_dataloader = DataLoader(train_dataset, batch_size=128, sampler=val_sampler, collate_fn=partial(split_batch, n_splits=n_splits))
test_dataloader = DataLoader(test_dataset, batch_size=64, collate_fn=partial(split_batch, n_splits=n_splits))

In [49]:
# Initialize your model
wandb.init(entity='ts-robustness', project='ml-course', config=config)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = TransformerClassification(config['model']).to(device)

# Initialize your optimizer and criterion
optimizer = build_optimizer(config, model)
criterion = nn.BCELoss()

def train_step(engine, batch):
    model.train()
    optimizer.zero_grad()
    x, y = batch[0].to(device), batch[1].to(device)
    y_pred = model(x)
    loss = criterion(y_pred, y.unsqueeze(1))
    loss.backward()
    optimizer.step()
    return loss.item()

trainer = Engine(train_step)

def validation_step(engine, batch):
    model.eval()
    with torch.no_grad():
        x, y = batch[0].to(device), batch[1].to(device)
        y_pred = model(x)
        return y_pred, y

train_evaluator = Engine(validation_step)
val_evaluator = Engine(validation_step)

# Attach metrics to the evaluators
metrics = {
    'accuracy': Accuracy(output_transform=lambda x: (x[0] > 0.5, x[1])),
    'loss': Loss(criterion, output_transform=lambda x: (x[0], x[1].unsqueeze(1)))
}

for name, metric in metrics.items():
    metric.attach(train_evaluator, name)

for name, metric in metrics.items():
    metric.attach(val_evaluator, name)

### Logging
@trainer.on(Events.ITERATION_COMPLETED)
def log_training_loss(trainer):
    batch_loss = trainer.state.output
    print("Training Results - Avg loss: {:.4f}".format(batch_loss))
    wandb.log({"train_loss": batch_loss})
    
@trainer.on(Events.EPOCH_COMPLETED)
def log_training_results(trainer):
    train_evaluator.run(train_dataloader)
    metrics = train_evaluator.state.metrics
    print("Training Results - Epoch: {}  Avg accuracy: {:.2f} Avg loss: {:.4f}"
          .format(trainer.state.epoch, metrics['accuracy'], metrics['loss']))
    wandb.log({"train_accuracy": metrics['accuracy'],
               "train_loss": metrics['loss']})

@trainer.on(Events.EPOCH_COMPLETED)
def log_validation_results(trainer):
    val_evaluator.run(val_dataloader)
    metrics = val_evaluator.state.metrics
    print("Validation Results - Epoch: {}  Avg accuracy: {:.2f} Avg loss: {:.4f}"
          .format(trainer.state.epoch, metrics['accuracy'], metrics['loss']))
    wandb.log({"val_accuracy": metrics['accuracy'],
               "val_loss": metrics['loss']})

# Run the training loop
trainer.run(train_dataloader, max_epochs=config['train']['n_epoch'])
wandb.finish()

Training Results - Avg loss: 0.6989
Training Results - Avg loss: 0.6869
Training Results - Avg loss: 0.7095
Training Results - Avg loss: 0.6974
Training Results - Avg loss: 0.7173
Training Results - Avg loss: 0.6907
Training Results - Avg loss: 0.6811
Training Results - Avg loss: 0.6987
Training Results - Avg loss: 0.6845
Training Results - Avg loss: 0.6907
Training Results - Avg loss: 0.6923
Training Results - Avg loss: 0.6826
Training Results - Avg loss: 0.6868
Training Results - Avg loss: 0.6806
Training Results - Avg loss: 0.6796
Training Results - Avg loss: 0.6654
Training Results - Avg loss: 0.6801
Training Results - Avg loss: 0.6739
Training Results - Avg loss: 0.7172
Training Results - Avg loss: 0.7069
Training Results - Avg loss: 0.6621
Training Results - Avg loss: 0.6704
Training Results - Avg loss: 0.7022
Training Results - Epoch: 1  Avg accuracy: 0.66 Avg loss: 0.6540
Validation Results - Epoch: 1  Avg accuracy: 0.61 Avg loss: 0.6661
Training Results - Avg loss: 0.6670
Trai

train_accuracy,▁▂▃▄▄▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇██▇███
train_loss,█▇█▇▇▆▆▆▅▄▄▅▃▃▃▄▃▂▂▂▃▂▂▂▂▃▂▃▂▁▂▂▃▁▁▁▁▂▁▁
val_accuracy,▁▃▄▅▅▆▇▇▇▇▇▇████▇▇████████████
val_loss,█▇▆▄▃▂▂▁▃▂▂▁▁▁▁▁▁▃▃▂▃▃▃▃▂▄▄▅▄▄
train_accuracy,0.9625
train_loss,0.11277
val_accuracy,0.84605
val_loss,0.56125


In [50]:
# Count the number of parameters in the transformer_encoder layer
transformer_encoder_params = sum(p.numel() for p in model.transformer_encoder.parameters())

# Count the number of parameters in the fc layer
fc_params = sum(p.numel() for p in model.fc.parameters())

print("Number of parameters in transformer_encoder:", transformer_encoder_params)
print("Number of parameters in fc layer:", fc_params)


Number of parameters in transformer_encoder: 434596
Number of parameters in fc layer: 25101
